In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source=dbutils.widgets.get("p_data_source")

#####Step 1 - Read the JSON file using the spark dataframe reader

In [0]:
constructor_schema="constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructor_df= spark.read.schema(constructor_schema).json(f"{raw_folder_path}/constructors.json")

#####Step 2 - Drop unwanted columns from the dataframe

In [0]:
from pyspark.sql.functions import col

In [0]:
#constructor_dropped_df=constructor_df.drop('url');
constructor_dropped_df=constructor_df.drop(col('url'));

#####Step 3 - Rename columns and add ingested data

In [0]:
from pyspark.sql.functions import current_timestamp,lit

In [0]:
constructor_droped_df=constructor_dropped_df.withColumnRenamed("constructorId","constructor_id").withColumnRenamed("constructorRef","constructor_ref") \
    .withColumn("datasource",lit(v_data_source))

In [0]:
constructor_final_df=add_ingestion_date(constructor_droped_df)

In [0]:
# display(constructor_final_df)

#####Step 4 - Write output to parquet file

In [0]:
constructor_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/constructors")

In [0]:
display(spark.read.parquet(f"{processed_folder_path}/constructors"))

In [0]:
dbutils.notebook.exit("success")